In [4]:
STUDENTS_PER_PROJECT = 4

In [2]:
ROSTER_FILE = "static/users_spring_2023.csv"
PROJECT_FILE = "static/projects_spring2023.csv"
BIDS_FILE = "static/bids_spring2023.csv"
FINAL_ALLOCS = "static/allocs_spring2023.csv"
LOG_FILE = "static/log"
ADD_STUDENTS = []
ADD_PROJECTS = []
STUDENTS_DID_NOT_BID = ['jl6258',
 'km3823',
 'ky2512',
 'yl5116',
 'zz3009',
 'dpk2132',
 'jz3531',
 'jc5678',
 'fa2603']
ADD_SEATS=["P033","P040","P007",
           "P032","P037","P008","P023","P010",
           "P026","P036","P024","P018","P009","P011"]



<h2>Demand Analytics from log file</h2>

In [ ]:
#USER: 2022-12-07 23:54:37.139154 him2109 ['P005', 5, 'P009', 1, 'P023', 990, 'P035', 1, 'P036', 1, 'P037', 1, 'P041', 1, 'P043', 1000]
logdata = dict()
projects = dict()
with open(LOG_FILE,"r") as f:
    for line in f:
        user = line.split()[3]
        choices = line.split('[')[1][:-2]
        logdata[user] = choices
with open(PROJECT_FILE,"r") as f:
    for line in f:
        projects[line.strip()] = 0
for user in logdata:
    allocs = logdata[user].split(", ")
    i=0
    while True:
        project = allocs[i]
        project=project[1:-1]
        alloc = int(allocs[i+1])
        projects[project]+=alloc
        i+=2
        if i >= len(allocs):
            break
print("Users: ",len(logdata))
project_totals = sorted([(k,v) for k,v in projects.items()],key=lambda x: -x[1])
for project in project_totals:
    print(project[0],project[1])

In [6]:
#Read class roster
def get_roster(file):
    roster_file = file
    student_list = list()
    with open(roster_file,'r') as f:
        for line in f:
            line = line.strip().split(',')
            uni = line[0]
            if uni == "Uni":
                continue
            if uni in STUDENTS_DID_NOT_BID:
                continue
            student_list.append(uni)    
    return student_list

len(get_roster(ROSTER_FILE))

176

In [ ]:
def get_projects(file):
    project_list = list()
    project_bids = dict()
    project_file = file
    with open(project_file,'r') as f:
        for line in f:
            line = line.strip().split('\t')
            project = line[0]
            if line[0] == 'PROJECT ID':
                continue
            project_list.append(project)
            project_bids[project] = STUDENTS_PER_PROJECT
    for project in ADD_SEATS:
            project_bids[project] += 1
    for p in ADD_PROJECTS:
        project_list.append(p)
        project_bids[p]=4

    return project_list,project_bids
get_projects(PROJECT_FILE)

In [ ]:
def get_bids(file):
    all_bids = list()
    bid_file = file
    with open(bid_file,'r') as f:
        for line in f:
            line = line.strip().split(',')
            bid_line = (line[0],line[1],int(line[2]))
            all_bids.append(bid_line)
    return all_bids
get_bids(BIDS_FILE)

In [ ]:
#Returns a list where each element is a pair (project,list of students with the same bid value)
#The projects are in decreasing order of bid values
def collect_bids_by_project_and_bid_amt(bids):
    bid_list = list()
    current_proj = ""
    for bid in bids: #active_bids:
        student = bid[0]
        proj = bid[1]
        bid_value = bid[2]
        if not proj == current_proj:
            if current_proj:
                bid_list.append(proj_list)
            proj_students = [student]
            proj_list = [proj,proj_students]

            current_proj = proj
        else:
            proj_students.append(student)
    bid_list.append(proj_list)
    return bid_list



In [ ]:
def do_allocations(bid_list,active_students,project_bids):
    import random
    all_allocations = list()
    for bid in bid_list:
        proj = bid[0]
        students = bid[1]
        seats = project_bids[proj]

        #skip if project is full
        if seats == 0:
            continue

        #remove already allocated students
        for student in students.copy():
            if not student in active_students:
                students.remove(student)

        count = len(students)                        
        if count <= seats: #allocate everyone. enough space
            for student in students:
                all_allocations.append((student,proj))
                active_students.remove(student)
                project_bids[proj] -= 1
        else:
            selected_students = random.sample(range(count),seats)
            for index in selected_students:
                all_allocations.append((students[index],proj))
                active_students.remove(students[index])
                project_bids[proj] -= 1
    return all_allocations,project_bids
           

In [ ]:
def missing_students(allocs,roster):
    missing=list()
    for s in roster:
        if s in [x[0] for x in allocs]:
            continue
        else:
            missing.append(s)
    return missing

In [ ]:
#TESTS
print("ROSTER: ",len(get_roster(ROSTER_FILE)))
projects,sizes = get_projects(PROJECT_FILE)
print("PROJECTS: ",len(projects),len(sizes))
bids = sorted(get_bids(BIDS_FILE),key=lambda x: (-x[2],x[1]))
print("BIDS: ",len(bids))
collected_bids_list = collect_bids_by_project_and_bid_amt(bids)
print("Project student collection: ",len(collected_bids_list))


In [ ]:
def get_total_utility(allocs,bids):
    total_utility = 0
    for alloc in allocs:
        student = alloc[0]
        project = alloc[1]
        for b in bids:
            if b[0]==student and b[1]==project:
                total_utility+=b[2]
    return total_utility

In [ ]:
counter = 0
results = list()
while True:
    roster = get_roster(ROSTER_FILE)
    project_list,project_seats = get_projects(PROJECT_FILE)
    all_bids = get_bids(BIDS_FILE)
    active_bids = sorted(all_bids,key=lambda x: (-x[2],x[1]))
    bid_list = collect_bids_by_project_and_bid_amt(active_bids)
    allocs,seats = do_allocations(bid_list,roster,project_seats) 
    missing = missing_students(allocs,roster)
    utility = get_total_utility(allocs,all_bids)
    counter +=1
    results.append((utility,len(missing)))
    if utility >= 144450:
        break
    #if utility >= 143101 and len(missing) <= 9:
    #    break
    #if len(missing) < 5:
    #if counter > 20000:
    #    break
#    if utility > 500:# and len(missing)<=2:
#        print(utility,len(missing),missing)
#        break
with open(FINAL_ALLOCS,'w') as f:
    for data in allocs:
        f.write(data[0]+'\t'+data[1]+'\t'+'\n')
    

In [ ]:
missing

In [ ]:
print(utility,len(missing))
print(max(results,key=lambda x: x[0]))
print(min(results,key=lambda x: x[1]))

In [ ]:
print(max(results,key=lambda x: x[0]))
print(min(results,key=lambda x: x[1]))

In [142]:
allocs.append((("sdj2129","P030"),
             ("cl4271","P033"),
             "ql2461","P040"))

TypeError: list.append() takes exactly one argument (4 given)

In [ ]:
def get_bid_demand(bids):
    demand = dict()
    for bid in bids:
        try:
            total = demand[bid[1]]
            demand[bid[1]] += bid[2]
        except:
            demand[bid[1]] = bid[2]
    return sorted([(key,value) for key,value in demand.items()],key=lambda x: -x[1])
#sum([x[1] for x in get_bid_demand(all_bids)])/1000
get_bid_demand(all_bids)

In [ ]:
def collect_bids_by_student_and_bid_amt(bids):
    bid_list = list()
    sorted_bids = sorted(bids,key=lambda x: (x[0],-x[2]))
    current_student = ""
    for bid in sorted_bids:
        student = bid[0]
        proj = bid[1]
        amt = bid[2]
        if not student == current_student:
            rank = 1
            p_amt = amt
            if current_student:
                bid_list.append(student_list)
            student_proj = [(proj,amt,rank)]
            student_list = [student,student_proj]
            current_student = student
        else:
            if not amt==p_amt:
                rank+=1
                p_amt=amt
            student_proj.append((proj,amt,rank))
    bid_list.append(student_list)
    return bid_list

def do_analysis(all_bids,allocs):
    ranks = list()
    s_b_dict = dict()
    for a,b in collect_bids_by_student_and_bid_amt(all_bids):
        s_b_dict.setdefault(a,[]).append(b)
    for a in allocs:
        s = a[0]
        p = a[1]
        for b in s_b_dict[s][0]:
            if b[0] == p:
                ranks.append((s,b[2]))
                break
    unique_ranks = {x[1] for x in ranks}
    prop=dict()
    for r in unique_ranks:
        pass
        prop[r] =  "%1.2f"%(len([x[1] for x in ranks if x[1]==r])/len(ranks)*100)

    return ranks,prop
#allocs.append(('yw3593','IB'))
ranks,props = do_analysis(all_bids,allocs)
props
#{1: '70.49', 2: '11.48', 3: '9.84', 4: '4.92', 5: '3.28'}

In [123]:
selected_ranks = list()
student_bids = dict()
for bid in active_bids:
    s = bid[0]
    proj = bid[1]
    if student_bids.get(s):
        student_bids[s].append(proj)
    else:
        student_bids[s]=[proj]
for alloc in allocs:
    s=alloc[0]
    b=student_bids[s]
    p=alloc[1]
    i=b.index(p)
    #print(s,i)
    selected_ranks.append((s,i))
from collections import Counter
ranks = [x[1] for x in selected_ranks]
Counter(ranks)

Counter({0: 130, 1: 25, 2: 7, 4: 4, 5: 1, 6: 3, 3: 2, 7: 1})

In [124]:
bid_totals = dict()
for bid in all_bids:
    project = bid[1]
    value = bid[2]
    bid_totals[project] = bid_totals.get(project,0)
    bid_totals[project] += value
    
{k: v for k, v in sorted(bid_totals.items(), key=lambda item: -item[1])}

#len(bid_totals)

{'P033': 24007,
 'P040': 22535,
 'P007': 17999,
 'P032': 16473,
 'P037': 15445,
 'P008': 14803,
 'P023': 13148,
 'P010': 12837,
 'P026': 12783,
 'P036': 10627,
 'P024': 10464,
 'P018': 10185,
 'P028': 8432,
 'P030': 8125,
 'P048': 8083,
 'P009': 7864,
 'P004': 7512,
 'P027': 7457,
 'P011': 7283,
 'P043': 7102,
 'P039': 6686,
 'P031': 6644,
 'P002': 6481,
 'P005': 5951,
 'P017': 5668,
 'P034': 5637,
 'P006': 5591,
 'P051': 4975,
 'P016': 4902,
 'P029': 4737,
 'P035': 4638,
 'P044': 4635,
 'P047': 4097,
 'P019': 4060,
 'P038': 3881,
 'P046': 3768,
 'P013': 3644,
 'P049': 3550,
 'P022': 3379,
 'P003': 3087,
 'P001': 2652,
 'P025': 2289,
 'P015': 1815,
 'P020': 1486,
 'P041': 1271,
 'P014': 1227,
 'P021': 789,
 'P042': 504,
 'P012': 160}

In [ ]:
def run_allocations():
    #Set random seed
    rand_seed = 42
    import random
    from random import randint
    random.seed = rand_seed

    roster,roster_dict = get_roster(ROSTER_FILE)
    project_list,project_seats = get_projects(PROJECT_FILE)
    all_bids = get_bids(BIDS_FILE)
    active_bids = sorted(all_bids,key=lambda x: (-x[2],x[1]))
    bid_list = collect_bids_by_project_and_bid_amt(active_bids)
    allocs,remaining_seats = do_allocations(bid_list,roster,project_seats)
    return allocs, remaining_seats,roster,roster_dict,all_bids

In [ ]:
while True:
    #allocs, remaining_seats,roster,roster_dict,all_bids = run_allocations()
    missing = missing_students(allocs,roster)
    count = len(missing)
    print(missing)
    if count == 2:
        break


In [ ]:

allocs

In [125]:
with open(FINAL_ALLOCS,'r') as f:
    projects = dict()
    for line in f:
        p=line.strip().split(',')[1]
        try:
            projects[p] +=1
        except:
            projects[p] = 1
sorted([(k,v) for k,v in projects.items()])

IndexError: list index out of range

In [135]:
allocs

[('hz2809', 'P001'),
 ('ms6538', 'P001'),
 ('kl3411', 'P002'),
 ('zq2177', 'P002'),
 ('ym2822', 'P002'),
 ('yc4125', 'P004'),
 ('jl6223', 'P004'),
 ('tc3210', 'P004'),
 ('qy2277', 'P005'),
 ('jy3249', 'P006'),
 ('qf2162', 'P006'),
 ('jt3368', 'P007'),
 ('al4372', 'P007'),
 ('fl2620', 'P007'),
 ('pa2636', 'P007'),
 ('xl3236', 'P007'),
 ('zt2310', 'P008'),
 ('jm5478', 'P008'),
 ('yd2662', 'P008'),
 ('qc2280', 'P008'),
 ('vwc2112', 'P008'),
 ('nr2861', 'P009'),
 ('id2378', 'P009'),
 ('rc3498', 'P009'),
 ('sz3096', 'P010'),
 ('ty2476', 'P010'),
 ('ml4812', 'P010'),
 ('gk2629', 'P010'),
 ('svs2141', 'P010'),
 ('xl3188', 'P011'),
 ('yl5093', 'P011'),
 ('nr2856', 'P011'),
 ('xl3228', 'P011'),
 ('sp4139', 'P013'),
 ('yz4306', 'P017'),
 ('zz2992', 'P017'),
 ('tv2299', 'P017'),
 ('zd2302', 'P017'),
 ('xg2397', 'P018'),
 ('sy3080', 'P018'),
 ('yy3256', 'P018'),
 ('wy2407', 'P018'),
 ('ry2447', 'P020'),
 ('xy2536', 'P023'),
 ('yd2658', 'P023'),
 ('zl3217', 'P023'),
 ('vs2809', 'P023'),
 ('ta2709',

In [141]:
len(allocs)

173

In [152]:
p_l,p_b = get_projects(PROJECT_FILE)
def get_open_seats(allocs,project_bids):
    project_allocs = dict()
    project_avail = dict()
    for thing in p_l:
        project_allocs[thing]=0
    for a in allocs:
        try:
            project_allocs[a[1]]+=1
        except:
            project_allocs[a[1]]=1
    for p in project_bids:
        if project_allocs[p] > project_bids[p]:
            project_bids[p] = project_allocs[p]
    for p in project_bids:
        project_avail[p] = project_bids[p]-project_allocs[p]
    return project_avail
proj_avail = get_open_seats(allocs,p_b)  
for p in proj_avail:
    if proj_avail[p]>0:
        print(p,proj_avail[p])


P001 1
P003 1
P012 4
P014 4
P015 2
P016 3
P020 2
P021 4
P025 3
P035 2
P041 3
P042 4
P047 2
P049 2


In [ ]:
for k in p_b:
    if not p_b[k] <= 0:
        print(k,p_b[k])

In [ ]:
def Convert(tup, di): 
    for a, b in tup: 
        di.setdefault(a, []).append(b) 
    return di 
di=dict()
x=Convert(collect_bids_by_student_and_bid_amt(all_bids),di)
x

In [133]:
counts = dict()
for p in projects:
    counts[p] = 0
for v in x.values():
    p = v[0][0][0]
    counts[p] += 1
print(counts)

NameError: name 'x' is not defined

In [ ]:
ranks

In [ ]:
with open(FINAL_ALLOCS,'w') as f:
    for data in allocs:
        f.write(data[0]+','+data[1]+'\n')
    

In [ ]:
def collect_bids_by_student_and_bid_amt(bids):
    bid_list = list()
    sorted_bids = sorted(bids,key=lambda x: (x[0],-x[2]))
    current_student = ""
    for bid in sorted_bids:
        student = bid[0]
        proj = bid[1]
        amt = bid[2]
        if not student == current_student:
            rank = 1
            p_amt = amt
            if current_student:
                bid_list.append(student_list)
            student_proj = [(proj,amt,rank)]
            student_list = [student,student_proj]
            current_student = student
        else:
            if not amt==p_amt:
                rank+=1
                p_amt=amt
            student_proj.append((proj,amt,rank))
    bid_list.append(student_list)
    return bid_list

#collect_bids_by_student_and_bid_amt(all_bids)

In [ ]:
collect_bids_by_student_and_bid_amt(all_bids)
all_bids

In [ ]:
missing_students(allocs,roster)

In [ ]:
missing_students(allocs,roster)

In [ ]:
allocs

In [ ]:
student_bids

In [144]:
allocs.extend((("sdj2129","P030"),("cl4271","P023"),("ql2461","P040")))

In [ ]:
student_bids['kw2874']

In [ ]:
all_bids

In [ ]:
def group_allocs_by_project(project_list,allocs):
    alloc_dict =dict()
    for p in project_list:
        alloc_dict[p] = list()
    for a in allocs:
        p = a[1]
        s = a[0]

        alloc_dict[p].append(s)
    
    return [",".join(v) + '"' for k,v in alloc_dict.items() ]
    return alloc_dict

group_allocs_by_project(project_list,allocs)

In [ ]:
[print(a[0],",",a[1],sep='') for a in allocs]

In [ ]:
bids

In [134]:
def highest_bids(projects,bids):
    counts = dict()
    for p in projects:
        counts[p] = 0
    for b in bids:
    

IndentationError: expected an indented block (2179670883.py, line 6)

In [9]:
def get_projects(file):
    project_list = list()
    project_bids = dict()
    project_file = file
    with open(project_file,'r') as f:
        for line in f:
            line = line.strip().split('\t')
            project = line[0]
            if line[0] == 'PROJECT ID':
                continue
            project_list.append(project)
            project_bids[project] = STUDENTS_PER_PROJECT
    for project in ADD_SEATS:
            project_bids[project] += 1
    for p in ADD_PROJECTS:
        project_list.append(p)
        project_bids[p]=4

    return project_list,project_bids

projects = get_projects(PROJECT_FILE)[0]
print(len(projects))
project_student_assignment=dict()
for project in projects:
    project_student_assignment[project] = list()

roster = get_roster(ROSTER_FILE)
student_dict = dict()
for student in roster:
    student_dict[student] = [student]
student_dict

with open("static/allocs_spring2023_final.csv",'r') as f:
    for line in f:
        student = line.strip().split(",")[0]
        project = line.strip().split(",")[1]
        if student == "UNI":
            continue
        print(student,project)
        project_student_assignment[project].append(student)


49
aa5001 P034
aa5009 P047
ae2756 P044
ah3354 P049
al4372 P007
am5927 P009
ask2289 P047
az2726 P031
ba2682 P022
bb3037 P038
bg2779 P048
bh2832 P020
bz2461 P028
cl4271 P033
cp3295 P025
cz2726 P004
dpk2132 P015
esl2170 P032
fa2603 P005
fl2620 P007
gk2629 P010
gs3204 P022
gz2343 P018
him2109 P043
hs3371 P016
hw2894 P013
hy2758 P036
hz2809 P001
id2378 P009
ih2396 P003
jg4537 P039
jl5268 P035
jl6223 P004
jl6245 P034
jl6248 P030
jl6259 P039
jm5478 P008
jt3368 P007
jw4264 P030
jy3242 P044
jy3248 P046
jy3249 P006
jy3259 P037
jz3489 P030
jz3505 P029
jz3531 P041
jz3533 P039
kc3421 P036
kd2867 P024
kj2586 P003
kk3585 P003
kl3411 P002
km3823 P003
kw3007 P024
ky2512 P015
lc3678 P026
ld3028 P009
lq2217 P026
lq2220 P032
lx2302 P040
lz2848 P027
lz2852 P051
mf3511 P028
mg4485 P013
mh4285 P031
ml4812 P010
ms6538 P001
mt3625 P001
mz2614 P040
mz2933 P043
nr2856 P011
nr2861 P009
nu2190 P001
nv2397 P036
ny2334 P029
nz2365 P002
pa2636 P007
pb2882 P047
ph2651 P019
pn2394 P049
qc2280 P008
qf2162 P006
qh2263 P0

In [10]:
project_student_assignment

{'P001': ['hz2809', 'ms6538', 'mt3625', 'nu2190'],
 'P002': ['kl3411', 'nz2365', 'ym2822', 'zq2177'],
 'P003': ['ih2396', 'kj2586', 'kk3585', 'km3823', 'seu2106'],
 'P004': ['cz2726', 'jl6223', 'tc3210', 'yc4125'],
 'P005': ['fa2603', 'qy2277', 'zz3009'],
 'P006': ['jy3249', 'qf2162', 'sd3585', 'zm2403'],
 'P007': ['al4372', 'fl2620', 'jt3368', 'pa2636', 'xl3236'],
 'P008': ['jm5478', 'qc2280', 'vwc2112', 'yd2662', 'zt2310'],
 'P009': ['am5927', 'id2378', 'ld3028', 'nr2861', 'rc3498', 'yt2821'],
 'P010': ['gk2629', 'ml4812', 'svs2141', 'sz3096', 'ty2476'],
 'P011': ['nr2856', 'xl3188', 'xl3228', 'yl5093', 'zz2972'],
 'P012': [],
 'P013': ['hw2894', 'mg4485', 'sp4139', 'zx2396'],
 'P014': [],
 'P015': ['dpk2132', 'ky2512'],
 'P016': ['hs3371', 'svr2120'],
 'P017': ['tv2299', 'yz4306', 'zd2302', 'zz2992'],
 'P018': ['gz2343', 'sy3080', 'wy2407', 'xg2397', 'yy3256'],
 'P019': ['ph2651', 'rb3623', 'sg4130'],
 'P020': ['bh2832', 'ry2447'],
 'P021': [],
 'P022': ['ba2682', 'gs3204', 'yl5122'